# Aqua 0.7 Operator Redesign
_8-Jan-19, donny@_

## Purpose
To improve the transparency, ease of understanding, and programming power of Aqua’s operator logic and usage. Specifically, to reconcile with the Terra operator hierarchy and make the Aqua algorithmic flow more visible, explicit, and extensible.

## Background: Motivation and Opportunities
The representation of matrices sparsely as linear combinations of Pauli operators is critical in many quantum algorithms. As such, the Operator classes are the workhorses of Aqua today (0.6.2), containing both the expectation value and evolution logic used by most of its algorithms.

However, there are several opportunities for improvement:
* **Basic Construction & Rapid Protoyping:** Aqua's Operators were initially built as procedural infrastructure rather than first-class programming primitives. Improvements to syntax and interfaces can enable the succinctness and power typical of mathematical Operator language
* **Separation of Operator Math and Operator Algorithms**
    * Ease of understanding: The "Operator algorithm" logic - the ExpectationValue, Evolution, grouping, and symmetry analysis - is mostly spread across the 3000-line operator hierarchy, and is very branchy for different modes of execution
    * Ease of extension: Modification to the expectation value, evolution, grouping, and symmetry logic is a core use case (e.g. the [CVaR expectation](https://arxiv.org/abs/1907.04769), [linear combination evolution](https://arxiv.org/abs/1202.5822), or the many recent papers on [Pauli grouping](https://www.nature.com/articles/nature23879)), but not explicitly supported today
* **Smooth Borders with Broader Qiskit**
    * Terra's `quantum_info` module also supports operator math, but is mostly matrix-based
    * **Remote Operator Algorithms:** Aer's fast ExpectationValue is not transparently or cleanly interchangeable with Aqua's local ExpectationValue today. The concept of an Algorithm not provided by Aqua is not yet defined to support this type of interchangeability cleanly

### Present State of Operators in Qiskit

Both Aqua and Terra include suites of modules to support Operator math, but do so very differently.

* Aqua
    * Operators are focused primarily on the procedural requirements of algorithmic execution
        * Modules are very large and include hundreds of lines of procedural algorithm code
        * Interfaces were not initial built for end-user usage as a programming primitive, and are therefore wordy and difficult for users to understand
        * Syntax is not built for rapid prototyping and lacks syntactic power of mathematical Operator language
    * Primarily focused on Pauli-basis Operators
        * WeightedPauli - $2^n\times 2^n$ Operators sparsely represented as complex combination of Paulis
        * MatrixOperator in the standard basis with $2^n\times 2^n$ elements was initially built for performance improvements which are no longer relevant
    * Only dependency on Terra is through Pauli module, but this is largely symbolic (not an inexorable component)
* Terra
    * Operator math is mostly built around QCVV and open Quantum systems modelling use cases
        * Support for Channel, Choi, Superoperator, Kraus, etc.
    * Operators are largely matrix-based and therefore do not support the Pauli-basis operations necessary to non-exponentially execute quantum algorithms
    * Used by: 
        * Aqua, 29 dependencies - Only Pauli module
        * Aer, 10 dependencies
        * Ignis, 2 dependencies

### Aqua Present Usage (0.6.2)

Within Aqua, the primary uses of Operators are:
* Qubit Observable (Hamiltonian, Cost Function, etc.) Construction
    * Used as sparse representations of large observables when constructing problems in Chemistry, Physics, Optimization, and Finance today
    * Also often a translation step between domain-specific problems and Quantum hardware-addressable equivalents
* ExpectationValues
    * Primarily used in VQE (and derivatives QAOA, UCCSD, etc.) as a device-executable cost function of the ansatz state
    * Expectation values can only be taken of Operators in the Pauli basis on Quantum hardware
    * Also present in the "Evolution of Hamiltonian" algorithm, which is simply state evolution by one operator followed by an expectation value by another operator
* State Evolution
    * Used in QPE (and derivatives AE, HHL, iQPE, etc.) as a Quantum circuit-representable matrix exponentiation
    * Used in UCCSD and QAOA ansatze and EOH algorithm as representation of system dynamics to simulate time evolution of a system on quantum hardware
    * Evolution can only be taken by Operators in the Pauli basis on Quantum hardware

#### Other Important Aqua Operator Features

* __Grouping__ - Grouping is a technique to reduce the number of circuit evaluations required to compute an ExpectationValue based on mutually commuting Paulis in the Operator decomposition.
* __Tapering__ - Tapering is a technique to remove qubits from a Hamiltonian of interest by identifying Z2 symmetries in the Hamiltonian.
* __Gradients__ - Many variational algorithms are improved dramatically when exact gradients of gate parameters with respect to the cost function observable are computed analytically rather than numerically. Aqua can compute these gradients and provide them to the optimizer directly.

### Aqua Present (0.6.2) Operator Object Model and Hierarchy

Aqua's Operators are organized as follows:
* `qiskit.aqua.operators`
    * base_operator.py: `BaseOperator(ABC)`
    * matrix_operator.py: `MatrixOperator(BaseOperator)`
    * weighted_pauli_operator.py: `WeightedPauliOperator(BaseOperator)`, __and__ `Z2Symmetries`
    * tpb_grouped_weighted_pauli_operator.py: `TPBGroupedWeightedPauliOperator(WeightedPauliOperator)`, essentially a wrapper around `WeightedPauliOperator` for backward compatibility.
    * pauli_graph: `PauliGraph`
    * op_converter.py: `to_weighted_pauli_operator(operator)`, `to_matrix_operator(operator)`, `to_tpb_grouped_weighted_pauli_operator(operator, grouping_func, **kwargs)`
    * common.py: Utility functions, inc. `evolution_instruction`, `pauli_measurement(circuit, pauli, qr, cr, barrier=False)`, `measure_pauli_z(data, pauli)`, `covariance(data, pauli_1, pauli_2, avg_1, avg_2)`, etc.
* `qiskit.chemistry` __OUT OF SCOPE__
    * fermionic_operator.py: `FermionicOperator`, contains `jordan_wigner`, `parity`, `bravyi_kitaev` Fermion-to-qubit operator mappings.
    * bksf.py: Another mapping
    * `.core`
        * chemistry_operator.py: `ChemistryOperator(ABC)`
        * hamiltonian.py: `Hamiltonian(ChemistryOperator)`

_Although many modules in optimization and finance construct Ising Hamiltonians as cost function observables, there is no Ising module._

### Terra Present (0.11.0) Operator Object Model and Hierarchy

Terra's Operators are organized as follows:
* `qiskit.quantum_info`
    * `.operators`
        * base_operator.py, pauli.py, operator.py (matrix operator), measures.py (`process_fidelity`), predicates.py (`is_unitary_matrix`, `is_hermitian_matrix`, `matrix_equal`, etc.), quaternion.py
        * `.channel`
            * quantum_channel.py (base), chi.py, choi.py, kraus.py, ptm.py, stinespring.py, superop.py, transformations.py
    * `.states`
        * quantum_state.py (base), densitymatrix.py, statevector.py, measures.py (`state_fidelity`), states.py (`basis_state`, `projector`, `purity`)
    * `.analysis`
        * average.py - ExpectationValue of diagonal operator
        * make_observable.py - Convert an observable in matrix form to dictionary form

## Requirements and Design

* Location and Ownership
* Object Hierarchy
    * Construction
    * Usage
    * Evaluation
    * Objects and Hierarchy
* Organization Options
* ExpectationValues
* Evolutions

### Location and Ownership in Qiskit

Given the presence of Operator logic in both Aqua and Terra, there are several options for their placement within Qiskit. The primary considerations here relate to which master branch tests them, who owns what in the case of breakage, and who owns what in the case of design.

In addition, some remote Operator algorithms are being discussed, with one already in production - the Aer Expectation Value. The location of these algorithms is also an important question.

#### Operator Location Considerations

* The Operator's centrality to Aqua means relying on an external library is a big overhead
    * Reliance on Terra has created frequent firedrills because behavior and interfaces change without integration testing
    * Firedrills are very difficult to troubleshoot because presently there is no integration testing between Terra and Aqua or design review to check whether a change will have downstream implications
    * Operator is so central to Aqua that it will require strong ownership by the Aqua team, constant maintenance and changes
* Centralized Operator primitives can simplify interfaces across Qiskit
    * By accepting a common Operator format derived from Terra, methods in different areas of Qiskit can communicate in a consistent format without dependencies
    * For example, Aer's expectation value can take a circuit and an Operator, rather than depend on Aqua to define its interface, or rely on an informal interface (e.g. lists) which must be validated
* Terra and Aqua's respective Operators can be delineated somewhat cleanly
    * Aqua and Terra's operators are seemingly used by completely different users for very different tasks (QA&A vs. QCVV or circuit analysis)
    * Terra's Operators are primarily matrix-based, while Aqua's are primarily composites of sparse representations (e.g. sums of Paulis or Circuits)
    * Though some are definitely shared, such as Pauli
* Operators and Gates may need to be reconciled at some point
    * The X, Y, and Z Paulis are not different from the X, Y, and Z Gates
    * Both the gate and operator models include functionality for converting unitary matrices to circuit operations

#### Operator Location Options

**A.** Move Aqua Operators into Terra, with:
1. Joint ownership by Aqua team
2. Aqua integration tests run on Terra's master branch (e.g. pulling in Aqua's master branch to execute tests)
3. Aligned release cycles so Aqua does not need to scramble to release when Terra does

**Big-A.** Combine Aqua and Terra into a single repo and jointly own Operators

**B.** Move all operators and states into Aqua, jointly owned by Terra team

**C.** Leave Operators split between Aqua and Terra, with dependency on Terra for some primitives (QuantumCircuit, Pauli), with joint ownership and Aqua integration testing

##### **Decision:** Following a discussion in Aqua Design Review, option **A** will be pursued for the remainder of this doc.

#### Provider-Specific Algorithm Location Options

**A.** Remote algorithms live in provider repo, and are tested and released at provider’s discretion

**B.** Remote algorithms live in Aqua, with Aqua integration testing of functionality in provider repo

**C.** Remote algorithms live in Aqua, with agreed upon interface to enforce consistency, and data interchange (e.g. an Operator format defined in Terra) tested in provider repo

### Object Model and Hierarchy 

What is an Operator _to a QA&A programmer?_

Ignoring the Physical definition of an Operator for a moment, as a _Quantum programming primitive,_ the Operator is:

* __Recursively defined__ - Operators can be one of several _primitives_ - e.g. Matrix, Pauli, Clifford, QuantumCircuit, or an arbitrary combination of these primitives, e.g. Addition, Tensor, Composition. 
    * It makes complete mathematical sense to add two primitives together, e.g. `(my_matrix+my_circuit)@my_pauli`. In classical programming, this would be like `5.7 + "pickle"`.
* __Both code and data__ - The Operator encodes both data (e.g. a matrix for eigensolution or a wavefunction being prepared) and computation (measure my wavefunction in this basis). There is little distinction between the two in Quantum programming.
* __Linear__ - The Operator is a recursively powerful construct, allowing algorithmic rearrangement not typically allowed in classical computation. 
    * `op1(op2(A,B)) == op1(op2(A)), op2(B))` in many cases, e.g. Expectation(A+B). 
    * The idea that `program(a*circuita + b*circuitb)` gives a mathematically valid result is highly surprising.
* __Algorithmically ubiquitous__ - Every quantum algorithm uses Operators. Algorithms are nearly always defined in literature by Operator operations. This language is rigorous, accepted, and compact. 
* __Eagerly Computable__ - In most cases, Operator computation can be partially compiled as parameters become available, allowing improved performance, functional modularity (e.g. passing a ready-to-run algorithm), and inspection transparency. For example:
    * A circuit can be compiled to a Qobj with parameters missing, to be filled in later
    * The full list of circuits necessary to execute an algorithm can be prepared pending some operator coefficients
    * A full algorithm can be prepared and passed to a user pending the insertion of some subcomponent (a choice of ExpectationValue algorithm) or parameters

#### Operator Definition: Primitives and Combinations

Operators can be _primitives_ or _combinations._ Primitives are base-level Operator representations which are not defined in terms of other primitives, but can be converted into one another with some computational work. Combinations are Operators which are constructed from functions of multiple primitives, such as sums and tensors. Combinations store the primitives from which they are constructed. Note that many Gates are present in other classes of primitives, and this must be reconciled as a follow-on to this redesign. The following should all be modules in the Operator hierarchy:

* Primitives
    * Matrix
    * Pauli - X, Y, Z, I
    * QuantumCircuit, Gate
        * Clifford
    * Projector - O, Z, P, M
        * Stabilizer
        * Graph State
        * QuantumCircuit
    * Others (follow-on): ZX, MPS, Dirac Matrix, Gell-Mann matrix
* Combinations
    * OpSum
    * OpComposition
    * OpKron
    * OpVec
    * OpExp
    * OpCombo (custom, user-defined recombination function)


#### Algorithms Definition: Primitives and Composites

Operations on Operators also can be described as primitives or combinations of such. Primitives are computations which can be performed directly on some available computation engine, such as Numpy or Quantum Hardware, while composites are constructed from piping primitives together. Algorithms accept only _specific primitives,_ so an algorithm taking a Pauli vs. one taking a matrix are fundamentally different, but are also defined over certain combinations of their input primitives. For example, a Change-of-Basis Expectation Value is defined to accept a Pauli and a Projector (or QuantumCircuit acting as one from Zero implicitly), but can also accept sums, tensors, and vectorizations of Paulis and Projectors. If an unsupported primitive, such as Matrix or OpComposition were passed in, an exception would be thrown.

* Primitives
    * Classical sum, product, tensor, trace, etc.
    * Z-Basis QuantumCircuit measurement / Trace (traditional QASM backend)
    * Primitive Conversion - Pauli to matrix, matrix to Pauli, etc.
    * Evolution Conversion - Trotter, Suzuki, etc.
    * Pauli Sum, Composition, Tensor
    * Change of Basis - Pauli, Fourier
    * Optimizers
    * External functions, such as Drivers or imports
* Composites
    * ExpectationValue
    * Existing Aqua Algorithms: VQE, QPE, HHL, etc.
    * Gradients

#### Parameterization and Eagerness

Operators and algorithms can be _parameterized,_ or missing some key information in order to execute. For Operators these may be sum coefficients, evolution times, QuantumCircuit parameters, and more. For Algorithms these may be input operators, execution parameters, or instances of algorithms used in computation which cannot be inferred by default (e.g. backend on which to execute, optimizer, etc.).

##### Eager Parameterization+Execution Interface Options:

An algorithm should execute as soon as it has filled the parameters necessary to do so. This is called **Eager Execution.** In a similar vein, OpSum can be seen as eagerly waiting for the contained operators to be summable, e.g. replaced with scalars by an expectation value. Some interface options for eagerness:

**Option A**: Algorithms should be **callable** with a parameter dictionary, triggering a breadth-first search to parameterize any sub-objects with the parameter dictionary. This may be too much hocus pocus and difficult for implementers of algorithms to understand. A user may want to parameterize without executing, so an `execute` parameter should be available in the parameterization function.

In [ ]:
my_op = Parameter('t1')*(Z^Z) + .6*(X^I)
my_vqe = VQE(backend=Parameter('backend'), 
             operator=my_op, 
             ansatz=Ry(qubits=2, reps=3), 
             optimizer=SLSQP(initial_point=Parameter('initial_rotations')))
my_vqe({'t1': .2j, 'backend': Aer.get_backend('qasm_simulator')}) 
    # Didn't return anything yet
rots = np.zeros(len(my_vqe.ansatz.params))
min_eig = my_vqe({'initial_rotations': rots})
    # Now a value is returned, and other execution information can be found inside the object

In [ ]:
    # Alternatively
my_vqe({'initial_rotations': rots}, execute=False)
min_eig = my_vqe()

**Option B:** Algorithms should have a `.run(param_dict)` method which accepts parameters and performs the breadth-first parameterization. The form factor of this would be similar to the above, but with `run()` instead of direct function calls. This has the benefit of some backward compatibility. 

**Option C:** Algorithms should support separate parameterization and execution functions. This is the most explicit, but is clunky in an eager execution regime, where execution is automatic if the algorithm is sufficiently parameterized.

All of an Algorithm or Operator's pending Parameters should be recursively returned by a `.params` function. _(Tentative)_ A `deepcopy` option should be available to return a deep copy of the algorithm with the desired parameterization, rather than parameterize the algorithm in-place (this is evaluated with `execute=False` by default).

##### Eager Partial Computation

Aqua should be **eager** in partial computation while some parameters necessary for execution are not yet available, to allow for inspection transparency and performance. For example, once backend information is available, circuits should be transpiled for the backend or otherwise prepared for execution. This can avoid many transpilations or preparations later if the circuits are duplicated for Operator composition, as in Change-of-Basis expectation values or gradients.

The choice of which partial computation to perform is left to the algorithm, so only worthwhile partial computations are performed. If parameters change, re-preparing the partial computation can be expensive, so a `lazy` parameter should be available in the callable function.

### Opportunity Example 1: Operator Construction and Manipulation

#### Present State

The center of Qiskit's algorithmic Operator logic is the WeightedPauli, being the only non-exponential scaling operator basis available today (the only other being the standard basis).

Qiskit supports several methods of WeightedPauli operator construction, none of which are self explanatory to a new user:

In [94]:
# from qiskit.quantum_info.operators import WeightedPauliOperator
from qiskit.aqua.operators import WeightedPauliOperator, MatrixOperator, op_converter
from qiskit.quantum_info.operators import Pauli

In [4]:
pauli_op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
    [.1j, Pauli.from_label('ZZ')],
])

In [5]:
pauli_op = WeightedPauliOperator.from_list(
    paulis=[Pauli.from_label('IX'),
            Pauli.from_label('ZY'),
            Pauli.from_label('ZZ')],
    weights=[.5, .2, .1j])

In [6]:
mat = [[0. +0.1j, 0.5-0.2j, 0. +0.j , 0. +0.j ],
       [0.5+0.2j, 0. -0.1j, 0. +0.j , 0. +0.j ],
       [0. +0.j , 0. +0.j , 0. -0.1j, 0.5+0.2j],
       [0. +0.j , 0. +0.j , 0.5-0.2j, 0. +0.1j]]
mat_op = MatrixOperator(mat)
pauli_op_from_mat = op_converter.to_weighted_pauli_operator(mat_op)
pauli_op == pauli_op_from_mat

True

Classical matrices can be exported for classical usage, again if the user already knows the Operator hierarchy somewhat well:

In [7]:
op_converter.to_matrix_operator(pauli_op).matrix.toarray()

array([[0. +0.1j, 0.5-0.2j, 0. +0.j , 0. +0.j ],
       [0.5+0.2j, 0. -0.1j, 0. +0.j , 0. +0.j ],
       [0. +0.j , 0. +0.j , 0. -0.1j, 0.5+0.2j],
       [0. +0.j , 0. +0.j , 0.5-0.2j, 0. +0.1j]])

Addition, scalar multiplication, composition, and deep equality are available for _Operators of the same representation_. However, composition uses the `*` operator, while Terra's operators and Python use `@`.

In [13]:
3*pauli_op + .2j*pauli_op == (3+.2j)*pauli_op

True

In [33]:
print((pauli_op * pauli_op).print_details())

II	(0.28+0j)
ZZ	0j
ZY	0j
IX	0j



In [9]:
pauli_op == mat_op

AttributeError: 'MatrixOperator' object has no attribute 'simplify'

#### [P0] Mathematical Construction

The above code lacks the mathematical structure or power of typical operator expressions, and lacks a self-explanatory relationship to the underlying Paulis used. As such, Qiskit should support the following (note that `^` is already implemented as the overload for kron product in Terra):

In [2]:
from qiskit.aqua.operators.pauli import X, Y, Z, I

In [9]:
op_new = .5*(I^X) + .2*(Z^Y) + .1j*(Z^Z)
op_new == pauli_op

True

#### [P1] Additional Math Overloads

The following overload operations are also desirable:
* Operator composition using `@` overload
    * __Decision:__ deprecate the `*` overload for composition?
* Power (`**3`), kronpower (`^3`)
* Partial Trace / Reordering - `Op[0:3]`, `Op[5:0:-2]`, `Op[(0,5,2,3,1)]`, `Op[:]` (full trace)

In [34]:
(pauli_op^2)**2 == (pauli_op^pauli_op)@(pauli_op^pauli_op)

True

Note that to support the above, the existing Pauli would need to support the following:
* Tensor, sum, and scalar multiplication which can return an OpSum, OpKron, or OpProduct
* Deep equality evaluation between operators
* A default set of simple operator instances

### Opportunity Example 2: Defining Algorithms by Operator Behavior

Over time, we have found that it is easiest to describe the behavior of Algorithms in terms of the flow of Operators through various components and subroutines. This description is naturally recursive, and considerably easier to understand than the present presentation of algorithmic flow in Aqua.

#### Present State - ExpectationValue Algorithms

Aqua's ExpectationValue is not contained within a single function or module, but rather split into several functions without a clear interface or flow for user usage. This is due to structural constraints in Aqua which are no longer present, where the algorithm requiring the expectation value held the backend object and could run circuits, but the operator could not. We encourage the reader to scan lines [361-395 of Aqua 6.1 VQE’s](https://github.com/Qiskit/qiskit-aqua/blob/stable/qiskit/aqua/algorithms/adaptive/vqe/vqe.py#L361) ExpectationValue calculation to try to understand where and how the expectation is computed. We’ve been asked by numerous Aqua users to explain how this code works, and most do not attempt to use it on their own.

The following is the shortest possible way to write an expectation value in Aqua. Note that it fundamentally requires the user to understand a certain execution flow, the correct functions to use to do this, and how those functions work with their execution mode. This takes a few hours to understand at least, often days. Further, there are no hints that tomography is being performed here, or matrix multiplication if the system chooses to do that instead.

In [4]:
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.variational_forms import RY
from qiskit.quantum_info import Pauli
from qiskit import BasicAer, execute, QuantumCircuit
from qiskit.circuit import Parameter
qasm_sim = BasicAer.get_backend('qasm_simulator')

In [2]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2j, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)
circuit.h([0,1])

evaluation_circuits = op.construct_evaluation_circuit(wave_function=circuit, statevector_mode=False)
result = execute(evaluation_circuits, qasm_sim).result()
expect, std = op.evaluate_with_result(result=result, statevector_mode=False)
expect

(0.5+0.005078125j)

#### Present State - State Evolution Algorithms

Evolution is somewhat more succinct, but more difficult to navigate in code.

In [4]:
from qiskit.circuit import Parameter

In [5]:
op = WeightedPauliOperator([
    [.5, Pauli.from_label('IX')],
    [.2, Pauli.from_label('ZY')],
])
circuit = QuantumCircuit(2)

θ = Parameter('θ')
instr = op.evolve_instruction(evo_time=θ)
circuit.append(instr, [0,1])
print(circuit.draw(fold=4000))
print('Decomposed:')
circuit.decompose().draw(fold=4000)

        ┌─────────────────┐
q_0: |0>┤0                ├
        │  Evolution^1(θ) │
q_1: |0>┤1                ├
        └─────────────────┘
Decomposed:


#### OperatorFlow Full VQE From Scratch Example (PENDING)

In [ ]:
ansatz = Ry(qubits=2, depth=3) @ (P^(-.1*O + 3*Z))
hamiltonian = 3*(I^Z) + .4j*(Z^Z)
expectation = CoBExpectation(initial_state_prep + ansatz, expectation)
print(expectation(np.zeroes(ansatz.num_params()))) # Starting expectation
gradient = ParamShiftGradient(expectation)

optimizer = AQGD(initial_point=np.zeroes(ansatz.num_params()))
VQE = AQGD(cost__fn=expectation, grad_fn=gradient)

### Other Benefits of OperatorFlow

* Obedient Eager Evaluation - Best of Eager and Lazy evaluation:
    * Partially evaluate whatever you can with the parameters you have
        * Allows transparency, inspection, rapid prototyping (e.g. Users couldn't find circuits or operator when working through JSON dictionaries)
        * Performance - partially compiled algorithms save massive amounts of compilation and deepcopy time
    * But not too early, not compiling preemptively for a possible parameter value
        * Objects can be returned without being totally incorrectly constructed for the next step or engine (e.g. building massive CNOT chains for UCCSD simulations)
        * Intractable but possible computations (e.g. convert to matrix and solve) are avoided
* Natural, Powerful, and Self-defining Programming Interfaces
    * __An algorithm's behavior is simply defined by the operator primitives it accepts and returns__
    * Nesting of algorithms is identical to user algorithm execution
* Ubiquitous parameters, and obvious interface for Optimization
    * OpCombo coefficients, primitive parameters, and algorithm parameters can all be parameterized
    * Algorithms of any level of completeness can be returned
    * Optimization is universal - simply pass a nearly-complete algorithm to an optimizer and the callable interface executes when the optimizer provides the parameters

## END ##

## ⚰️⚰️⚰️⚰️⚰️⚰️ Graveyard ⚰️⚰️⚰️⚰️⚰️⚰️

### Organization and Hierarchy

Option A. Leave Aqua's operators separate from Terra, focusing on Quantum Algorithms & Applications (QA&A) usage and constructing the Operator computation flow infrastructure

Option B. Move Aqua's operators into Terra, including a large refactor to support the Operator computation flow and and an extended basis hierarchy, with deep integration tests between Terra and Aqua and strict SLAs

Option C. Move Aqua's operators into Terra on a limited basis, mirroring existing functionality only and with no refactoring to improve fit, interface, or function

Option Big-C: Move Aqua into Terra Repo and jointly maintain the Operators

#### Other Usage Syntactic Sugar

In addition, the following operations are desirable:
* `to_matrix()` - Method to allow quick access to unscalable classical tools, e.g. numpy eigensolution
* `op.basis_change_operator(origin_basis="Zn")` - Returns an operator to change the basis from `origin_basis` to basis of `op`
* `to_quantum_circuits` - returns a single or list of quantum circuits which represent composition of the operator with a quantum circuit, including complex summation later
* Trace, Partial Trace, Determinant, Norms, Adjoints - Where possible, linear algebra should be easily accessible 

#### Future Work: Other Important Qubit Operator Bases, MixedOperator

* Other Operator bases - Operators can also be constructed as complex combinations of Cliffords, Clifford+Ts, Projectors, Stabilizers, QuantumCircuits, and ZX Hamiltonians. Qiskit should support these primitives to allow more powerful Operator compositions. Generalizations of the Pauli basis, including the Dirac and Gell-Mann matrices, may be worth consideration.
* MixedOperator - Qiskit should consider supportting a MixedOperator, holding a complex combination of various operators.

#### Grouping

Aqua's grouping functionality is only relevant to ExpectationValues today.

In [92]:
qaoa_cost_op = WeightedPauliOperator([
    [.5, Pauli.from_label('ZIZ')],
    [.2, Pauli.from_label('ZZI')],
    [.1j, Pauli.from_label('IZZ')],
])

grouped_cost_op = TPBGroupedWeightedPauliOperator.sorted_grouping(qaoa_cost_op)
grouped_cost_op._basis

[(Pauli(z=[True, True, True], x=[False, False, False]), [0, 1, 2])]

#### Tasks Not Possible Today

* Explicitly using Aer's ExpectationValue, or explicitly disabling it
* Modifying the ExpectationValue, Evolution, grouping, or symmetry code without copying a significant amount of code

### WeightedPauliOperator Not Available in Terra

Terra contains operator utilities in the `qiskit.quantum_info.operators` directory, focused primarily on the mathematical operations between matrix-represented operators of various types (e.g. Choi, Pauli, etc.). Terra's operators do not contain much of the Pauli-basis logic in Aqua today, and are not interoptable with Aqua's operator algorithms. As such, these utilities are only accessible to Aqua users.

### Ignis and Aer Logic Living in Aqua

* Aqua's ExpectationValue relies on a form of tomography, which in principle should live within Ignis's tomography utilities.
* Aer's ExpectationValue is a super-superuser feature today
    * The interface (and existence) of the Aer-provided algorithm is not made obvious because there is no way to specify a non-Aqua-provided algorithm in Qiskit
    * In Aqua, there is no simple way to specify which ExpectationValue algorithm the user wants. 
    * Aer's ExpectationValue is woven throughout the core operator code in a way that is branchy, inexorable, and difficult for users to understand and control.

## Core Requirements / Interface

1. Broadly Available (Qiskit-wide) WeightedPauli Logic
    1. Basic Construction and Usage
    1. Interface, Variants, Organization
        1. Option A: WeightedPauli class in Terra
        1. Option B: WeightedUnitary class in Terra
    1. Integration Testing
1. ExpectationValue and Evolution algorithm hierarchies
    1. Interface, Variants, Organization
        1. Usage of Operator Abstractions within Aqua Algorithms
        1. Usage of Operator Abstractions without Aqua Algorithms
    1. ExpectationValue Reconciliation with Ignis tomography
        1. Option A: Leave ExpectationValue in Aqua
        1. Option B: Keep ExpectationValue algo in Aqua, move logic into Ignis
        1. Option C: Move ExpectationValue algo into Ignis
1. Aer ExpectationValue and Evolution
    1. Option I: Leave AerExpectation in Aqua
    1. Option II: Central Qiskit Algorithms base classes
    1. Option III: Merge Aqua and Terra
1. Smooth Borders with Applications Primitives - e.g. FermionicOperator
    1. Move Operator Translation Into Application Stacks, Consistently

### Broadly Available (Qiskit-wide) WeightedPauli Logic

A **WeightedPauliOperator** is an n-qubit operator represented sparesely as a complex combination of n-qubit Pauli operators (tensor products of n single-qubit Paulis). All hermitian operators can be represented in this basis.

Many quantum algorithms depend on this sparse representation of operators in the Pauli basis. Terra contains a suite of Matrix-represented operators within the quantum_info module, including rich logic for combining and manipulating various operator types, but does not include a WeightedPauli operator. 
* The Aqua operator does not need to replicate Terra's logic within MatrixOperator, and should instead rely on Terra for Matrix-based operator logic. 
* Aqua is the not the only place in Qiskit which requires WeightedPauli Operators. These are used across quantum information, and should be interoptable with the other existing Terra operators.

**Requirement:** The mathematical (non-algorithmic) Operator logic in Aqua should be moved into Terra, and Aqua should use these as its core operator objects.

### Breaking out the ExpectationValue and Evolution to be explicit modules which can be interchanged and extended.

Users want to easily find and understand ExpectationValue and Evolution logic, and understand the algorithmic variations available to them.

...

In [ ]:
from qiskit.aqua.algorithms import ExpectationValue, Evolution

avg = ExpectationValue(operator, backend).run(circuit)
instr = Evolution(operator, backend).instruction(param)

### Terse and transparent Operator usage within the Aqua algorithms, characteristic of end-user usage of the same algorithms.

...

In [ ]:
class VQE(QuantumAlgorithm):
    def __init__(self, operator, var_form, optimizer,
                 initial_point=None, backend=backend, callback=None, ...):
        ...
        self._expectation_value = ExpectationValue(self._operator, self._backend)
    
    def _energy_evaluation(self, params):
        circuits = self._var_form.construct_circuit(params)
        energy, stdev = self._expectation_value.run(circuits)
        return energy

### Explicit modules for Aer's ExpectationValue and Evolution which can be swapped for Aqua's, and a framework for remote backends to provide RemoteAlgorithm modules without depending on Aqua. 

...

In [ ]:
from qiskit.providers.aer import AerExpectationValue

my_vqe = VQE()

### Reconciling the implicit tomography in Aqua's expectation value with Ignis's tomography functionality to avoid code duplication and allow generic access to the functionality.

...

### Clear and smooth borders with the Operator-related objects in Aqua, such as the FermionicOperator and Hamiltonian in Chemistry, and the proposed LadderOperator and OptimizationProblem.

...

## Technical Design

* Changes to Terra Operator Hierarchy
* Aqua Algorithm Hierarchy
    * Groups and Symmetries Options
* Changes to Ignis
* Qiskit Remote Algorithms
* Aer ExpectationValue and Evolution
* IBMQ or other backend RemoteAlgorithms

### Changes to Terra Operator Hierarchy

There are two options for the migration of the Operator mathematical logic into Terra's quantum_info:

#### Option A: Move WeightedPauliOperator into quantum_info and refactor

...

#### Option B: Add Linear Combination functionality to Operator to allow for WeightedPauli, WeightedClifford, and Combinations

* e.g. Pauli a + Pauli b = LinComb(a, b)
* Circuit Operator?

...

...

## Timeline and Gameplan

...

## Future Extension Opportunities